In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
from netCDF4 import num2date, date2num, date2index
import datetime as dt
import pandas as pd
from StringIO import StringIO
import xarray as xr

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
import re

def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [3]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')

In [4]:
allURL = []
allURL.append(jason2URL)
allURL.append(jason3URL)
allURL.append(saralURL)
allURL.append(sentinel3aURL)
allURL.append(cryosat2URL)
allURL.append(enviURL)
allURL.append(geosatURL)
allURL.append(ersURL)
allURL.append(gfoURL)
allURL.append(topURL)

In [5]:
boxLat = []
boxLon = []
boxWh = []
boxT = []
boxQ = []

In [6]:
#Cape Melville coordinates
latmin = -14.0
latmax = -13.0  
lonmin = 145.0
lonmax = 146.0

if latmin>latmax:
    print('Error wrong definition of min and max lat!!!')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon!!!')

In [7]:
# 
start_date = dt.datetime(1985,1,1)
end_date = dt.datetime(2019,2,21)

#start_date = dt.datetime(2010,1,1)
#end_date = dt.datetime(2010,12,31)


In [8]:
for u in range (len(allURL)):
    urlON = allURL[u]
    for k in range (len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
       
        if u == 2:
            wh = ncs.variables['SWH_KA_CAL'][:]
            qc = ncs.variables['SWH_KA_quality_control'][:]
        
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]
            qc = ncs.variables['SWH_KU_quality_control'][:]

        # Get desired time interval  
        time_var = ncs.variables['TIME']
        tt = ncs.variables['TIME'][:]
        
        timing = netCDF4.num2date(tt,time_var.units)
        
        #data in correct time, quality flag and lat/lon
        for k in range(len(timing)):
            if timing[k] >= start_date and timing[k] <= end_date:
                if lats[k]>latmin and lats[k]<latmax and lons[k]>lonmin and lons[k]<lonmax:
                    boxLat.append(lats[k])
                    boxLon.append(lons[k])
                    boxWh.append(wh[k])
                    boxT.append(tt[k])
                    boxQ.append(qc[k])

In [9]:
len(boxT)
boxT

[8586.981900860788,
 8586.981912615709,
 8586.98192455154,
 8586.98193630646,
 8586.98194806138,
 8586.981959997094,
 8586.981971752015,
 8586.981983506936,
 8586.981995261856,
 8591.477658510557,
 8591.477670355933,
 8591.477682201192,
 8591.477693956112,
 8591.477705801488,
 8591.477717556409,
 8591.477729401784,
 8591.47774124716,
 8591.47775300208,
 8591.47776484734,
 8591.47777660226,
 8596.897552806768,
 8596.897564561688,
 8596.897576316609,
 8596.897588252323,
 8596.897600007243,
 8596.897611762164,
 8596.897623517085,
 8596.897635452799,
 8596.89764720772,
 8601.393300419557,
 8601.393312174478,
 8601.393324019853,
 8601.393335774774,
 8601.393347620033,
 8601.393359465408,
 8601.393371220329,
 8601.393383065704,
 8601.393394820625,
 8601.393406666,
 8601.393418420805,
 8601.39343026618,
 8606.813198423013,
 8606.813210177934,
 8606.813221932855,
 8606.813233868685,
 8606.813245623605,
 8606.813257378526,
 8606.81326931424,
 8606.81328106916,
 8606.813292824081,
 8611.30894042

In [10]:
for k in range(len(boxLat)):
    if k == 0:
        lat = boxLat[k]
        lon = boxLon[k]
        wh = boxWh[k]
        tt = boxT[k]
        qc = boxQ[k]
    else:
        lat = np.append(lat,boxLat[k])
        lon = np.append(lon,boxLon[k])
        wh = np.append(wh,boxWh[k])
        tt = np.append(tt,boxT[k])
        qc = np.append(qc,boxQ[k])

In [11]:
tt[0]
boxT
tt

array([8586.98190086, 8586.98191262, 8586.98192455, ..., 6455.12111966,
       6455.12113211, 6455.12114454])

In [12]:
print(netCDF4.num2date(tt,time_var.units))

[datetime.datetime(2008, 7, 5, 23, 33, 56, 234372)
 datetime.datetime(2008, 7, 5, 23, 33, 57, 249997)
 datetime.datetime(2008, 7, 5, 23, 33, 58, 281253) ...
 datetime.datetime(2002, 9, 4, 2, 54, 24, 738280)
 datetime.datetime(2002, 9, 4, 2, 54, 25, 814456)
 datetime.datetime(2002, 9, 4, 2, 54, 26, 888671)]


In [13]:
ids = np.where(np.logical_and(wh>0,qc==1))
whpos = wh[ids]
qc_ = qc[ids]
lon_ = lon[ids]
tt_ = tt[ids]
lat_ = lat[ids]

print len(whpos), 'whpos'
print len(qc_),'qc_'
print len(lon_),'lon_'
print len(tt_),'tt_'
print len(lat_),'lat_'

34802 whpos
34802 qc_
34802 lon_
34802 tt_
34802 lat_


In [61]:
tt_

array([8586.98190086, 8586.98191262, 8586.98192455, ..., 6455.12111966,
       6455.12113211, 6455.12114454])

In [18]:
days = [] #daystart
daye = [] #dayend
for k in range(len(whpos)):
    t1 = netCDF4.num2date(tt_[k],time_var.units)
    if k == 0: #get day 0. Then else: every day after that
        days.append(0)
        dd = netCDF4.num2date(tt_[k],time_var.units)
        it = 0
    else:
        if t1.day != dd.day: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            daye.append(k-1)
            days.append(k)
            it += 1
            dd = netCDF4.num2date(tt_[k],time_var.units)

In [ ]:
ids = np.where(np.logical_and(wh>0,qc==1))
whpos = wh[ids]
qc_ = qc[ids]
lon_ = lon[ids]
tt_ = tt[ids]
lat_ = lat[ids]

print len(whpos), 'whpos'
print len(qc_),'qc_'
print len(lon_),'lon_'
print len(tt_),'tt_'
print len(lat_),'lat_'

In [15]:
print 'mean whpos', np.mean(whpos)
print 'max whpos',max(whpos)
print 'min whpos', min (whpos)

print 'min lon',min(lon_)
print 'min lon',min(lon_)
print 'max lon',max(lon_)

print 'min lat', min(lat_)
print 'max lat',max(lat_)

print min(netCDF4.num2date(tt_,time_var.units))
print max (netCDF4.num2date(tt_,time_var.units))

mean whpos 1.6800049060284783
max whpos 4.949000358581543
min whpos 0.003000000026077032
min lon 145.00002
min lon 145.00002
max lon 145.99998
min lat -13.999892
max lat -13.000028
1985-04-05 21:27:22.624997
2019-02-20 11:53:11.406250


In [16]:
len(whpos)
print min(netCDF4.num2date(tt_,time_var.units))
print max(netCDF4.num2date(tt_,time_var.units))

1985-04-05 21:27:22.624997
2019-02-20 11:53:11.406250


In [17]:
for k in range(len(whpos)):
    print netCDF4.num2date(tt_[k],time_var.units)

2008-07-05 23:33:56.234372
2008-07-05 23:33:57.249997
2008-07-05 23:33:58.281253
2008-07-05 23:33:59.296878
2008-07-05 23:34:00.312503
2008-07-05 23:34:01.343749
2008-07-05 23:34:02.359374
2008-07-05 23:34:03.374999
2008-07-05 23:34:04.390624
2008-07-10 11:27:49.695312
2008-07-10 11:27:50.718753
2008-07-10 11:27:51.742183
2008-07-10 11:27:52.757808
2008-07-10 11:27:53.781249
2008-07-10 11:27:54.796874
2008-07-10 11:27:55.820314
2008-07-10 11:27:56.843754
2008-07-10 11:27:57.859380
2008-07-10 11:27:58.882810
2008-07-10 11:27:59.898435
2008-07-15 21:32:28.562505
2008-07-15 21:32:29.578130
2008-07-15 21:32:30.593755
2008-07-15 21:32:31.625001
2008-07-15 21:32:32.640626
2008-07-15 21:32:33.656251
2008-07-15 21:32:34.671876
2008-07-15 21:32:35.703122
2008-07-15 21:32:36.718747
2008-07-20 09:26:21.156250
2008-07-20 09:26:22.171875
2008-07-20 09:26:23.195315
2008-07-20 09:26:24.210940
2008-07-20 09:26:25.234371
2008-07-20 09:26:26.257811
2008-07-20 09:26:27.273436
2008-07-20 09:26:28.296877
2

2009-03-25 06:49:38.160159
2009-03-25 06:49:39.179687
2009-03-25 06:49:40.199214
2009-03-25 06:49:41.222655
2009-03-25 06:49:42.242183
2009-03-25 06:49:43.261720
2009-03-25 06:49:44.281248
2009-03-30 16:54:10.781248
2009-03-30 16:54:11.796873
2009-03-30 16:54:12.828129
2009-03-30 16:54:13.843754
2009-03-30 16:54:14.859379
2009-03-30 16:54:15.890625
2009-03-30 16:54:16.906250
2009-03-30 16:54:17.921875
2009-03-30 16:54:18.937500
2009-03-30 16:54:19.968746
2009-04-04 04:48:02.972651
2009-04-04 04:48:03.996092
2009-04-04 04:48:05.015630
2009-04-04 04:48:06.035157
2009-04-04 04:48:08.074223
2009-04-04 04:48:09.093751
2009-04-04 04:48:10.117191
2009-04-04 04:48:11.136719
2009-04-04 04:48:12.156247
2009-04-04 04:48:13.175784
2009-04-04 04:48:14.195312
2009-04-04 04:48:15.214840
2009-04-09 14:52:42.812500
2009-04-09 14:52:43.828125
2009-04-09 14:52:44.843750
2009-04-09 14:52:45.874996
2009-04-09 14:52:46.890621
2009-04-09 14:52:47.906246
2009-04-09 14:52:48.937502
2009-04-09 14:52:49.953127
2

2010-08-08 11:40:30.953122
2010-08-12 23:34:19.406254
2010-08-12 23:34:20.421879
2010-08-12 23:34:21.437504
2010-08-12 23:34:22.453130
2010-08-12 23:34:23.484375
2010-08-12 23:34:24.500000
2010-08-12 23:34:25.515626
2010-08-12 23:34:26.546871
2010-08-12 23:34:27.562496
2010-08-12 23:34:28.578121
2010-08-12 23:34:29.593746
2010-08-12 23:34:30.625002
2010-08-18 09:38:58.898438
2010-08-18 09:38:59.921879
2010-08-18 09:39:00.937504
2010-08-18 09:39:01.960934
2010-08-18 09:39:02.984375
2010-08-18 09:39:04
2010-08-18 09:39:05.023440
2010-08-18 09:39:06.039065
2010-08-18 09:39:07.062496
2010-08-22 21:32:51.312502
2010-08-22 21:32:52.343748
2010-08-22 21:32:53.359373
2010-08-22 21:32:54.374998
2010-08-22 21:32:56.421879
2010-08-22 21:32:57.437504
2010-08-22 21:32:58.453129
2010-08-22 21:32:59.484375
2010-08-22 21:33:00.500000
2010-08-22 21:33:01.515625
2010-08-22 21:33:02.546871
2010-08-28 07:37:30.328125
2010-08-28 07:37:31.343750
2010-08-28 07:37:32.367190
2010-08-28 07:37:33.390621
2010-08-

2011-08-04 22:41:24.109375
2011-08-04 22:41:25.125000
2011-08-10 08:45:51.968752
2011-08-10 08:45:52.984377
2011-08-10 08:45:54.007817
2011-08-10 08:45:57.062498
2011-08-10 08:45:59.109379
2011-08-10 08:46:00.125004
2011-08-10 08:46:01.148434
2011-08-14 20:39:44.343749
2011-08-14 20:39:45.359374
2011-08-14 20:39:46.390620
2011-08-14 20:39:47.406245
2011-08-14 20:39:48.421870
2011-08-14 20:39:49.437496
2011-08-14 20:39:50.468751
2011-08-14 20:39:51.484376
2011-08-14 20:39:52.500002
2011-08-14 20:39:53.531247
2011-08-14 20:39:54.546872
2011-08-14 20:39:55.562498
2011-08-20 06:44:23.371090
2011-08-20 06:44:24.390628
2011-08-20 06:44:25.410155
2011-08-20 06:44:26.429683
2011-08-20 06:44:27.449221
2011-08-20 06:44:28.468749
2011-08-20 06:44:29.492189
2011-08-20 06:44:30.511717
2011-08-20 06:44:31.531255
2011-08-24 18:38:15.250003
2011-08-24 18:38:16.265628
2011-08-24 18:38:17.281253
2011-08-24 18:38:18.296878
2011-08-24 18:38:19.328124
2011-08-24 18:38:20.343749
2011-08-24 18:38:21.359374
2

2012-08-15 17:45:16.500004
2012-08-15 17:45:17.515630
2012-08-15 17:45:18.531255
2012-08-15 17:45:19.562500
2012-08-15 17:45:20.578126
2012-08-15 17:45:21.593751
2012-08-21 03:49:49.667972
2012-08-21 03:49:50.687500
2012-08-21 03:49:51.707028
2012-08-21 03:49:52.726566
2012-08-21 03:49:53.749996
2012-08-21 03:49:54.769534
2012-08-21 03:49:55.789062
2012-08-21 03:49:56.808589
2012-08-21 03:49:57.828127
2012-08-25 15:43:42.093754
2012-08-25 15:43:43.109379
2012-08-25 15:43:44.140625
2012-08-25 15:43:45.156250
2012-08-25 15:43:46.171875
2012-08-25 15:43:47.187500
2012-08-25 15:43:48.218746
2012-08-25 15:43:49.234371
2012-08-25 15:43:50.249996
2012-08-25 15:43:51.281252
2012-08-25 15:43:52.296877
2012-08-25 15:43:53.312502
2012-08-31 01:48:21.856446
2012-08-31 01:48:22.876949
2012-08-31 01:48:23.897462
2012-08-31 01:48:24.916990
2012-08-31 01:48:25.937504
2012-08-31 01:48:26.957031
2012-08-31 01:48:27.977535
2012-08-31 01:48:28.998048
2012-08-31 01:48:30.017576
2012-09-04 13:42:14.156247
2

...

1996-10-23 08:09:21.476560
1996-10-23 08:09:22.554687
1996-10-27 20:03:07.171871
1996-10-27 20:03:08.249998
1996-10-27 20:03:09.312505
1996-10-27 20:03:10.390622
1996-10-27 20:03:11.468750
1996-10-27 20:03:12.531246
1996-10-27 20:03:13.609374
1996-10-27 20:03:14.687501
1996-10-27 20:03:15.765628
1996-10-27 20:03:16.828125
1996-10-27 20:03:17.906252
1996-11-02 06:07:45.710942
1996-11-02 06:07:46.785157
1996-11-02 06:07:47.859371
1996-11-02 06:07:48.933596
1996-11-02 06:07:50.007811
1996-11-02 06:07:51.078123
1996-11-02 06:07:52.152348
1996-11-02 06:07:53.226562
1996-11-02 06:07:54.300777
1996-11-06 18:01:38.203129
1996-11-06 18:01:39.265625
1996-11-06 18:01:40.343753
1996-11-06 18:01:41.421880
1996-11-06 18:01:42.484377
1996-11-06 18:01:43.562504
1996-11-06 18:01:44.640621
1996-11-06 18:01:45.718749
1996-11-06 18:01:46.781245
1996-11-06 18:01:47.859373
1996-11-06 18:01:48.937500
1996-11-12 04:06:16.914066
1996-11-12 04:06:17.988280
1996-11-12 04:06:19.062495
1996-11-16 16:00:09.515626
1

1998-04-01 20:51:10.156254
1998-04-01 20:51:11.218750
1998-04-01 20:51:12.296878
1998-04-01 20:51:13.375005
1998-04-01 20:51:14.437502
1998-04-01 20:51:15.515629
1998-04-06 08:44:59.531247
1998-04-06 08:45:00.601559
1998-04-06 08:45:01.671871
1998-04-06 08:45:02.749998
1998-04-06 08:45:03.820311
1998-04-06 08:45:04.898438
1998-04-06 08:45:05.968750
1998-04-06 08:45:07.039062
1998-04-06 08:45:08.117189
1998-04-06 08:45:09.187501
1998-04-06 08:45:10.265629
1998-04-06 08:45:11.335941
1998-04-11 18:49:38.984375
1998-04-11 18:49:40.062502
1998-04-11 18:49:41.140630
1998-04-11 18:49:42.203126
1998-04-11 18:49:43.281254
1998-04-11 18:49:44.359371
1998-04-11 18:49:45.421878
1998-04-11 18:49:46.499995
1998-04-11 18:49:47.578122
1998-04-16 06:43:32.734380
1998-04-16 06:43:33.808595
1998-04-16 06:43:34.878907
1998-04-16 06:43:35.953121
1998-04-16 06:43:37.027346
1998-04-16 06:43:38.101561
1998-04-16 06:43:39.175785
1998-04-16 06:43:40.246097
1998-04-16 06:43:41.320312
1998-04-16 06:43:42.394527
1

1999-09-19 07:31:28.367183
1999-09-19 07:31:29.445310
1999-09-19 07:31:30.515622
1999-09-19 07:31:31.593750
1999-09-19 07:31:32.664062
1999-09-19 07:31:33.734374
1999-09-19 07:31:34.812501
1999-09-19 07:31:35.882813
1999-09-23 19:25:19.062498
1999-09-23 19:25:20.125004
1999-09-23 19:25:21.203122
1999-09-23 19:25:22.281249
1999-09-23 19:25:23.343746
1999-09-23 19:25:24.421873
1999-09-23 19:25:25.500000
1999-09-23 19:25:26.562497
1999-09-23 19:25:27.640624
1999-09-23 19:25:28.718752
1999-09-23 19:25:29.781248
1999-09-23 19:25:30.859376
1999-09-29 05:29:57.964844
1999-09-29 05:29:59.039058
1999-09-29 05:30:00.113283
1999-09-29 05:30:01.183595
1999-09-29 05:30:02.257810
1999-09-29 05:30:03.332034
1999-09-29 05:30:04.406249
1999-09-29 05:30:05.480464
1999-09-29 05:30:06.554688
1999-10-09 03:28:27.757816
1999-10-09 03:28:28.832031
1999-10-09 03:28:29.902343
1999-10-09 03:28:30.976568
1999-10-09 03:28:32.050782
1999-10-09 03:28:33.124997
1999-10-09 03:28:34.199222
1999-10-09 03:28:35.273436
1

2001-01-01 20:16:05.296880
2001-01-11 18:14:25.296876
2001-01-11 18:14:26.375003
2001-01-11 18:14:27.437500
2001-01-11 18:14:28.515627
2001-01-11 18:14:29.593754
2001-01-11 18:14:30.656251
2001-01-11 18:14:31.734378
2001-01-11 18:14:32.812496
2001-01-11 18:14:33.875003
2001-01-11 18:14:34.953130
2001-01-11 18:14:36.031247
2001-01-21 16:12:56.781249
2001-01-21 16:13:03.781251
2001-01-21 16:13:04.781246
2001-01-27 02:17:36.439455
2001-01-27 02:17:37.513670
2001-01-27 02:17:38.585933
2001-01-27 02:17:39.660158
2001-01-27 02:17:40.732421
2001-01-27 02:17:41.806636
2001-01-27 02:17:42.880861
2001-01-27 02:17:43.953124
2001-01-27 02:17:45.027349
2001-01-31 14:11:28.656246
2001-01-31 14:11:29.718753
2001-01-31 14:11:30.796870
2001-01-31 14:11:31.874997
2001-01-31 14:11:32.937504
2001-01-31 14:11:34.015621
2001-01-31 14:11:35.093749
2001-01-31 14:11:36.156246
2001-01-31 14:11:37.234373
2001-01-31 14:11:38.312500
2001-01-31 14:11:39.390627
2001-01-31 14:11:40.453124
2001-02-06 00:16:08.280033
2

2002-03-09 15:17:06.703122
2002-03-09 15:17:07.781249
2002-03-09 15:17:08.843746
2002-03-09 15:17:09.921873
2002-03-09 15:17:11
2002-03-14 03:10:56.335939
2002-03-14 03:10:57.408202
2002-03-14 03:10:58.482417
2002-03-14 03:10:59.556642
2002-03-14 03:11:00.628905
2002-03-14 03:11:01.703130
2002-03-14 03:11:02.775393
2002-03-14 03:11:03.849608
2002-03-14 03:11:04.923833
2002-03-14 03:11:05.996096
2002-03-14 03:11:07.070311
2002-03-14 03:11:08.144535
2002-03-19 13:15:35.828123
2002-03-19 13:15:36.906251
2002-03-19 13:15:40.125002
2002-03-19 13:15:42.265626
2002-03-19 13:15:43.343753
2002-03-19 13:15:44.406250
2002-03-24 01:09:28.997071
2002-03-24 01:09:30.070310
2002-03-24 01:09:31.143559
2002-03-24 01:09:32.216798
2002-03-24 01:09:33.291013
2002-03-24 01:09:34.364262
2002-03-24 01:09:35.438477
2002-03-24 01:09:36.510740
2002-03-24 01:09:37.584965
2002-03-24 01:09:38.659179
2002-03-24 01:09:39.732418
2002-03-29 11:14:07.484373
2002-03-29 11:14:08.562500
2002-03-29 11:14:09.632812
2002-03-

In [19]:
print days[0],daye[0]


0 8


In [20]:
for t in range(len(days)-1): 
    print np.mean(lon_[days[t]:days[t+1]]) #check if this is right? Length btn dates.
    print len(lon_[days[t]:days[t+1]])
    

145.9107
9
145.88956
11
145.9127
9
145.89076
12
145.9126
9
145.89105
11
145.91144
10
145.8936
12
145.91203
9
145.88947
12
145.90868
9
145.88644
9
145.91554
9
145.87112
11
145.88634
7
145.89256
12
145.91351
9
145.88629
11
145.91248
10
145.88405
13
145.90443
9
145.8772
12
145.90462
9
145.88477
12
145.90842
10
145.87755
12
145.90587
10
145.89278
12
145.9361
7
145.82388
5
145.88486
4
145.89099
12
145.91093
9
145.88919
5
145.91653
5
145.95515
4
145.89021
5
145.89561
11
145.91211
10
145.88759
11
145.90544
9
145.88158
10
145.90656
10
145.8882
11
145.90628
10
145.9035
11
145.84505
4
145.8756
11
145.90985
9
145.88399
13
145.87122
3
145.87976
13
145.90082
10
145.88387
13
145.89989
10
145.88391
12
145.91185
9
145.8918
9
145.90361
9
145.88484
12
145.9068
9
145.88576
12
145.90538
9
145.88873
11
145.91174
10
145.97304
3
145.91087
9
145.88503
12
145.91238
10
145.88905
12
145.90762
10
145.88129
10
145.90926
6
145.8755
11
145.90643
10
145.89043
12
145.90623
10
145.8984
10
145.9141
9
145.87802
12
145.91

145.91287
9
145.88777
12
145.91068
9
145.88542
12
145.90256
10
145.87907
12
145.9039
10
145.87718
12
145.90471
10
145.88275
11
145.91144
9
145.87776
12
145.91043
9
145.88329
12
145.90482
9
145.89458
12
145.91183
9
145.88225
12
145.94563
6
145.88606
12
145.88933
5
145.90918
8
145.91125
8
145.80042
4
145.91286
9
145.8843
11
145.8884
7
145.90784
10
145.88243
13
145.90121
6
145.89827
11
145.89001
16
145.402
16
145.17108
16
145.88998
16
145.40912
16
145.17674
16
145.88904
16
145.40881
16
145.16992
16
145.88899
16
145.39272
11
145.18979
12
145.88632
16
145.42163
16
145.17097
16
145.88463
17
145.4114
16
145.16568
16
145.89624
14
145.42558
17
145.17584
16
145.88611
14
145.4194
16
145.17648
16
145.89143
15
145.4255
14
145.17963
15
145.93776
6
145.40811
16
145.16367
17
145.89339
14
145.41327
16
145.1734
16
145.90858
11
145.41693
16
145.17331
14
145.8867
16
145.41562
16
145.16867
16
145.87386
16
145.40463
15
145.17265
16
145.89102
16
145.41678
15
145.17337
16
145.87018
14
145.4391
13
145.17352
16

145.17303
3
145.89032
15
145.41017
16
145.16495
15
145.87737
15
145.41399
16
145.17117
15
145.89032
15
145.40953
16
145.87964
15
145.4004
14
145.17375
15
145.88486
14
145.40855
16
145.16641
15
145.88092
15
145.41577
16
145.17012
15
145.88013
15
145.40784
16
145.16832
14
145.92273
5
145.42169
15
145.17482
15
145.90736
12
145.42856
15
145.14816
11
145.88318
8
145.41693
16
145.16087
13
145.88683
15
145.4266
15
145.18184
8
145.87993
15
145.40874
14
145.15785
13
145.4191
15
145.17094
15
145.87544
15
145.40927
15
145.15883
15
145.87398
15
145.41116
15
145.1648
15
145.42412
15
145.88316
15
145.4214
15
145.17236
15
145.85342
10
145.41917
15
145.1704
15
145.8846
15
145.4149
15
145.15701
5
145.88547
8
145.41939
15
145.1384
2
145.88177
15
145.17133
15
145.88292
15
145.40765
15
145.16937
15
145.88216
15
145.43602
11
145.17003
15
145.8831
15
145.40337
14
145.20276
11
145.88397
15
145.41696
14
145.13544
9
145.88336
15
145.41449
16
145.17888
14
145.88705
15
145.37312
3
145.17058
15
145.87971
15
145.4

11
145.90799
9
145.88753
12
145.91254
8
145.89218
11
145.919
9
145.9363
1
145.91301
8
145.88237
11
145.91861
9
145.88206
11
145.91376
9
145.88593
11
145.90738
9
145.8471
6
145.88249
12
145.89706
8
145.92073
7
145.88344
11
145.90967
8
145.88298
11
145.91318
8
145.88809
10
145.89682
8
145.88362
6
145.88408
12
145.91127
9
145.90503
9
145.88274
11
145.88756
11
145.9075
9
145.88307
11
145.90622
9
145.88586
11
145.91219
9
145.8829
11
145.91595
9
145.8915
11
145.91736
9
145.88086
12
145.92325
8
145.88025
11
145.91138
9
145.87793
12
145.91573
9
145.89998
9
145.87868
12
145.91693
9
145.88672
12
145.90904
9
145.88037
12
145.91547
9
145.8892
12
145.91669
9
145.90765
9
145.88756
12
145.91243
8
145.88744
12
145.91354
9
145.88927
12
145.90292
8
145.94846
5
145.92677
7
145.88454
12
145.90515
9
145.90631
9
145.89514
3
145.91302
8
145.88744
12
145.91742
9
145.89497
11
145.90384
9
145.89735
8
145.9082
9
145.88062
11
145.9156
9
145.88785
11
145.90848
10
145.88055
12
145.88855
8
145.83902
8
145.90543
9
14

In [22]:
#Practice

meanwh = [] #mean wave height from each satellite track

for t in range(len(days)-1):
    meanwh.append(np.mean(whpos[days[t]:days[t+1]]))


In [23]:
for k in range(len(meanwh)):
    if k==0:
        mwh = meanwh[k]
    else:
        mwh = np.append(mwh,meanwh[k])

In [59]:
mwh[1]

3.179545619271018

In [58]:
for k in range(days[1],days[2]):
    
    print netCDF4.num2date(mwh[k],time_var.units),whpos[k]
    
print days[1],daye[1]

1985-01-04 03:04:04.811440 3.0900001525878906
1985-01-04 07:19:21.609650 3.308000087738037
1985-01-03 14:37:26.413422 3.188000202178955
1985-01-03 04:16:09.610977 3.123000144958496
1985-01-02 09:29:58.697246 3.1740002632141113
1985-01-02 18:23:39.436046 3.0860002040863037
1985-01-02 18:37:19.207649 3.1560001373291016
1985-01-02 10:24:19.205246 3.182000160217285
1985-01-03 09:34:49.320582 3.136000156402588
1985-01-02 16:46:59.526443 3.2710001468658447
1985-01-02 14:13:55.207074 3.261000156402588
9 19


Dont think I need this

In [ ]:

for k in range(days[1],days[2]):
    
    print netCDF4.num2date(tt_[k],time_var.units),whpos[k]
    
print days[1],daye[1]

In [ ]:
for k in range(days[2],days[3]):
    
    print netCDF4.num2date(tt_[k],time_var.units),whpos[k]
    
print days[2],daye[2]

In [ ]:
sorted1 = np.sort(tt_)
for k in range(len(sorted1)):
    print netCDF4.num2date(sorted1[k],time_var.units)

In [ ]:
##EXTRACT DATA BY MONTH

In [30]:
#find time of large wave heights
months = []
for k in range(len(tt_)):
    if whpos[k]>4.5:
        dt = netCDF4.num2date(tt_[k],time_var.units)
        months.append(dt.month)

In [31]:
print len(months)
print months
months

15
[7, 7, 7, 7, 7, 5, 3, 3, 3, 3, 3, 6, 8, 7, 7]


[7, 7, 7, 7, 7, 5, 3, 3, 3, 3, 3, 6, 8, 7, 7]

In [ ]:
sorted1.shape

USE THIS FOR MONTHS :)

In [ ]:
#find all january wave heights. USE tt_, as it will then call all other variables
janWh = []
febWh = []
marWh = []
aprWh = []
mayWh = []
junWh = []
julWh = []
augWh = []
sepWh = []
octWh = []
novWh = []
decWh = []

for k in range(len(tt_)):
    dt = netCDF4.num2date(tt_[k],time_var.units)
    if dt.month == 1:
        janWh.append(k)
    if dt.month == 2:
        febWh.append(k)
    if dt.month == 3:
        marWh.append(k)
    if dt.month == 4:
        aprWh.append(k)
    if dt.month == 5:
        mayWh.append(k)
    if dt.month == 6:
        junWh.append(k)
    if dt.month == 7:
        julWh.append(k)
    if dt.month == 8:
        augWh.append(k)
    if dt.month == 9:
        sepWh.append(k)
    if dt.month == 10:
        octWh.append(k)
    if dt.month == 11:
        novWh.append(k)
    if dt.month == 12:
        decWh.append(k)

In [ ]:
janW = whpos[janWh]
febW = whpos[febWh]
marW = whpos[marWh]
aprW = whpos[aprWh]
mayW = whpos[mayWh]
junW = whpos[junWh]
julW = whpos[julWh]
augW = whpos[augWh]
sepW = whpos[sepWh]
octW = whpos[octWh]
novW = whpos[novWh]
decW = whpos[decWh]


print 'Month       Min               Max                 Mean'
print 'Jan', janW.min(),janW.max(),janW.mean()
print 'Feb', febW.min(),febW.max(),febW.mean()
print 'Mar', marW.min(),marW.max(),marW.mean()
print 'Apr', aprW.min(),aprW.max(),aprW.mean()
print 'May', mayW.min(),mayW.max(),mayW.mean()
print 'Jun', junW.min(),junW.max(),junW.mean()
print 'Jul', julW.min(),julW.max(),julW.mean()
print 'Aug', augW.min(),augW.max(),augW.mean()
print 'Sep', sepW.min(),sepW.max(),sepW.mean()
print 'Oct', octW.min(),octW.max(),octW.mean()
print 'Nov', novW.min(),novW.max(),novW.mean()
print 'Dec', decW.min(),decW.max(),decW.mean()

In [ ]:
janW = np.asarray(janWh)
janW.min(),janW.max(),janW.mean()

In [ ]:
julyW = np.asarray(julyWh)
julyW.min(),julyW.max(),julyW.mean()

In [ ]:
julyW